In [ ]:
import pandas as pd
import tensorflow as tf
import os
import datetime
import numpy as np

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split


# Train

In [ ]:
# Load the training data
train_data = pd.read_csv('input_data/train_data.csv')

# Prepare the training data
commands = train_data.iloc[:, 0].values
brightness = train_data.iloc[:, 1].astype(int).values

# Normalize brightness values to [0, 1]
y_normalized = brightness / 100.0

# Tokenize the commands
tokenizer = Tokenizer()
tokenizer.fit_on_texts(commands)
sequences = tokenizer.texts_to_sequences(commands)

# Pad the sequences to ensure uniform input size
max_sequence_length = max(len(seq) for seq in sequences)
X_train = pad_sequences(sequences, maxlen=max_sequence_length)

# Load the validation data
val_data = pd.read_csv('input_data/val_data.csv')

# Prepare the validation data
val_commands = val_data.iloc[:, 0].values
val_brightness = val_data.iloc[:, 1].astype(int).values

# Normalize brightness values to [0, 1]
y_val_normalized = val_brightness / 100.0

# Tokenize the validation commands
val_sequences = tokenizer.texts_to_sequences(val_commands)
X_val = pad_sequences(val_sequences, maxlen=max_sequence_length)

# Create the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64, input_length=max_sequence_length))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))  # Dropout layer to reduce overfitting
model.add(LSTM(32))
model.add(Dropout(0.2))  # Another dropout layer
model.add(Dense(1, activation='sigmoid'))  # Change to sigmoid if normalizing

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
model.fit(
    X_train,
    y_normalized,
    epochs=15,
    validation_data=(X_val, y_val_normalized),  # Add validation data
    callbacks=[tensorboard_callback]
)

# Save the model
model.save('output_data/led_brightness_model.keras')

# Run

In [ ]:
model = load_model('output_data/led_brightness_model.keras')

# Make predictions on the validation data
predictions = model.predict(X_val)

# Denormalize predictions to [0, 100] and round to two decimal places
predicted_brightness = (predictions * 100.0).flatten()
predicted_brightness_rounded = np.round(predicted_brightness, 2)

# Calculate the difference between true brightness and predicted brightness
brightness_difference = val_brightness - predicted_brightness_rounded

# Display the results
results = pd.DataFrame({
    'Command': val_commands,
    'True Brightness': val_brightness,
    'Predicted Brightness': predicted_brightness_rounded,
    'Difference': brightness_difference
})

print(results)


# Export trained data for Arduino or ESP32 boards

In [ ]:
output_dir = os.path.join('LED_Controller', 'data')
os.makedirs(output_dir, exist_ok=True)

# Load the model
model = tf.keras.models.load_model('output_data/led_brightness_model.keras')

# Check the model architecture
model.summary()


# Extract parameters (weights and biases for each layer)
weights = []
biases = []
for layer in model.layers:
    if 'dense' in layer.name:
        w, b = layer.get_weights()
        weights.append(w)
        biases.append(b)
# Save parameters as C++ arrays
for i, (w, b) in enumerate(zip(weights, biases)):
    weights_path = os.path.join(output_dir, f'dense_{i}_weights.h')
    biases_path = os.path.join(output_dir, f'dense_{i}_biases.h')
    np.savetxt(weights_path, w.flatten(), delimiter=',', header=f'Dense {i} weights', comments='')
    np.savetxt(biases_path, b.flatten(), delimiter=',', header=f'Dense {i} biases', comments='')

print(f"Parameters were successfully saved to the folder: {output_dir}")